# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Решение:

In [133]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [134]:
KEYWORDS = ['python', 'парсинг', 'fedora', 'windows']

In [135]:
page = requests.get('https://habr.com/ru/all/')

In [136]:
soup = BeautifulSoup(page.text, 'html.parser')
articles = soup.find_all('article', class_='post')

In [140]:
def parse_date(a_date):
    '''
    Функция разбирает строку вида "сегодня в 11:05" или "2 октября 2020 в 13:24"
    И возвращает дату (без времени) в объекте типа pandas.Timestamp
    '''
    if a_date.find('сегодня') != -1:
        return pd.to_datetime('today').date()
    elif a_date.find('вчера') != -1:
        return pd.to_datetime('today').date() - pd.to_timedelta('1D')
    else:
        date_list = a_date.strip().split(' ')
        date_list
        if date_list[1] == 'января':
            month = 1
        elif date_list[1] == 'февраля':
            month = 2
        elif date_list[1] == 'марта':
            month = 3
        elif date_list[1] == 'апреля':
            month = 4
        elif date_list[1] == 'мая':
            month = 5
        elif date_list[1] == 'июня':
            month = 6
        elif date_list[1] == 'июля':
            month = 7
        elif date_list[1] == 'августа':
            month = 8
        elif date_list[1] == 'сентября':
            month = 9
        elif date_list[1] == 'октября':
            month = 10
        elif date_list[1] == 'ноября':
            month = 11
        elif date_list[1] == 'декабря':
            month = 12
        return pd.to_datetime(date_list[2]+'-'+str(month)+'-'+date_list[0]).date()

### Стандартный скрипт:

In [149]:
results = pd.DataFrame()
for article in articles:
    for keyword in KEYWORDS:
        if article.find('div', class_='post__text').text.lower().find(keyword) != -1:
            results = pd.concat([results, pd.DataFrame([{'title': article.h2.text.strip(), 'link': article.h2.a['href'], 'date': parse_date(a_date)}])])
            break
results

,title,link,date
0,Готов релиз Fedora Linux для смартфонов,https://habr.com/ru/company/selectel/blog/521976/,2020-01-21
0,Вторая жизнь Virtual Floppy Drive,https://habr.com/ru/post/521962/,2020-01-21


### Улучшенный скрипт:

In [153]:
import time
results = pd.DataFrame()
for article in articles:
    article_page = requests.get(article.h2.a['href'])
    article_soup = BeautifulSoup(article_page.text, 'html.parser')
    article_text = article_soup.find('div', class_='post__text').text
    for keyword in KEYWORDS:
        if article_text.lower().find(keyword) != -1:
            results = pd.concat([results, pd.DataFrame([{'title': article.h2.text.strip(), 'link': article.h2.a['href'], 'date': parse_date(article.find('span', class_='post__time')), 'text': article_text.strip()}])])
            break
    time.sleep(0.2)
results

,title,link,date,text
0,Готов релиз Fedora Linux для смартфонов,https://habr.com/ru/company/selectel/blog/521976/,2020-10-04,Скриншот десктопной версии Fedora Linux\r\nLin...
0,Вторая жизнь Virtual Floppy Drive,https://habr.com/ru/post/521962/,2020-10-04,Когда-то давно у меня была коллекция старинных...
0,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/digital-ecosystems...,2020-10-04,"В этом дайджесте — обновления Android, собстве..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

## Решение:

In [208]:
EMAIL = ['test@test.ru', 'test@test.com', 'test2@test.ru']

In [209]:
r_data = {'emailAddresses':EMAIL}
r_headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast',
             'Vaar-Version':'0'}
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
post_req = requests.post(url, json = r_data, headers = r_headers)
post_req

<Response [200]>

In [210]:
import json

In [211]:
response = json.loads(post_req.text)

In [303]:
breaches = pd.DataFrame.from_dict(response['breaches'], orient='index').reset_index()
breaches.rename(columns={'index':'breaches'}, inplace=True)
breaches['breaches'] = breaches['breaches'].astype(int)
breaches.head()

,breaches,site,recordsCount,description,publishDate
0,3873,expressshop.lv,33655,"In March 2017, ExpressShop's user database was...",2017-04-20T00:00:00Z
1,3399,mobango.com,692332,"In August 2011, Mobango's user database was al...",2017-03-15T00:00:00Z
2,347,aristos.pw,189277,"In September 2016, Aristos.pw was breached. Th...",2017-01-16T00:00:00Z
3,13302,transylvania.ru,18124,"At an unconfirmed date, Transylvania.ru's user...",2018-01-07T00:00:00Z
4,3762,policeone.com,715127,"In 2015, PoliceOne's forum was an alleged targ...",2017-04-10T00:00:00Z


In [313]:
breaches_by_mail = pd.DataFrame()
for mail, values in response['summary'].items():
    current = pd.DataFrame.from_dict(values)
    current['mail'] = mail
    breaches_by_mail = pd.concat([breaches_by_mail, current])
    breaches_by_mail['breaches'] = breaches_by_mail['breaches'].astype(int)

In [314]:
breaches_by_mail = breaches_by_mail.merge(breaches, on='breaches', how='left')

In [315]:
breaches_by_mail = breaches_by_mail[['mail', 'publishDate', 'site', 'description']]
breaches_by_mail.tail(10)

,mail,publishDate,site,description
33,test@test.ru,2017-03-15T00:00:00Z,casinos.microgaming.com,At an unconfirmed date Casinos.Microgaming.com...
34,test@test.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
35,test@test.ru,2017-02-06T00:00:00Z,umoveindia.com,"In June 2016, UMoveIndia's user database was b..."
36,test@test.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
37,test@test.ru,2017-04-04T00:00:00Z,evony.com,"In July 2016, gaming site Evony's user databas..."
38,test@test.ru,2017-04-14T00:00:00Z,frozencraft.ru,"In February 2015, Frozen Craft's user database..."
39,test@test.ru,2017-09-28T00:00:00Z,talk.oneland.su,"In December 2016, Oneland's user database was ..."
40,test2@test.ru,2018-01-07T00:00:00Z,flashgamesnexus.com,"In 2017, Flash Games Nexus' user database was ..."
41,test2@test.ru,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
42,test2@test.ru,2017-03-01T00:00:00Z,asterios.ru,"In June 2010, Asterios' user database was alle..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

## Решение:

In [352]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
GROUP = 'netology'
TOKEN = ''
VERSION = '5.103'

In [353]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}

In [354]:
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [355]:
response = json.loads(res.text)

In [356]:
news = pd.DataFrame.from_dict(response['response']['items'])

In [357]:
news = news[['date', 'text']]

In [358]:
news.head(10)

,date,text
0,1601280010,Всё ещё дарим подарки 🎁\n \nКаждую неделю разы...
1,1601819640,Собрали для вас подборку выступлений с конфере...
2,1601796240,*партнёрский пост* \n \nИнтернет-издание «Лайк...
3,1601729640,В новой афише интересных онлайн-мероприятий: ч...
4,1601710920,"Повышаем конверсию, составляем динамические пи..."
5,1601650440,"В эфире #пятничнаябеседка. Здесь мы обсуждаем,..."
6,1601623320,"Точно решили, кем хотите стать в диджитале, но..."
7,1601562240,"Можно придумать взрывную идею и ждать, когда о..."
8,1601539680,Ещё несколько лет назад все ловили покемонов н...
9,1601478480,"Новая задачка: давайте узнаем, кто из дизайнер..."
